In [1]:
text = "This is some text"
byte_array = bytearray(text, 'utf-8')
print(byte_array)

bytearray(b'This is some text')


In [3]:
ids = list(byte_array)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


In [5]:
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")

[1212, 318, 617, 2420]

In [6]:
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")

0: !
1: "
2: #
3: $
4: %
5: &
6: '
7: (
8: )
9: *
10: +
11: ,
12: -
13: .
14: /
15: 0
16: 1
17: 2
18: 3
19: 4
20: 5
21: 6
22: 7
23: 8
24: 9
25: :
26: ;
27: <
28: =
29: >
30: ?
31: @
32: A
33: B
34: C
35: D
36: E
37: F
38: G
39: H
40: I
41: J
42: K
43: L
44: M
45: N
46: O
47: P
48: Q
49: R
50: S
51: T
52: U
53: V
54: W
55: X
56: Y
57: Z
58: [
59: \
60: ]
61: ^
62: _
63: `
64: a
65: b
66: c
67: d
68: e
69: f
70: g
71: h
72: i
73: j
74: k
75: l
76: m
77: n
78: o
79: p
80: q
81: r
82: s
83: t
84: u
85: v
86: w
87: x
88: y
89: z
90: {
91: |
92: }
93: ~
94: �
95: �
96: �
97: �
98: �
99: �
100: �
101: �
102: �
103: �
104: �
105: �
106: �
107: �
108: �
109: �
110: �
111: �
112: �
113: �
114: �
115: �
116: �
117: �
118: �
119: �
120: �
121: �
122: �
123: �
124: �
125: �
126: �
127: �
128: �
129: �
130: �
131: �
132: �
133: �
134: �
135: �
136: �
137: �
138: �
139: �
140: �
141: �
142: �
143: �
144: �
145: �
146: �
147: �
148: �
149: �
150: �
151: �
152: �
153: �
154: �
155: �
156: �
157: �
158:

## 1.2 Building the Vocabulary

The goal of the BPE tokenization algorithm is to build a vocabulary of commonly occurring subwords, such as:

- `298: ent` (which can be found in *entangle*, *entertain*, *enter*, *entrance*, *entity*, etc.).

## 1.3 BPE Algorithm Outline

### 1. Identify Frequent Pairs

- In each iteration, scan the text to find the most commonly occurring pair of bytes (or characters).

### 2. Replace and Record

- Replace that pair with a new placeholder ID (one not already in use).
  - Example: If we start with `0...255`, the first placeholder would be `256`.
- Record this mapping in a lookup table.
- The size of the lookup table is a **hyperparameter**, also called the **"vocabulary size"**.
  - (For GPT-2, this is **50,257**.)

### 3. Repeat Until No Gains

- Keep repeating steps **1 and 2**, continually merging the most frequent pairs.
- Stop when no further compression is possible (e.g., no pair occurs more than once).

### 4. Decompression (Decoding)

- To restore the original text, **reverse the process** by substituting each ID with its corresponding pair using the lookup table.

---

## 1.4 BPE Algorithm Example

### 1.4.1 Concrete Example of the Encoding Part (Steps 1 & 2)

#### **Suppose we have the text (training dataset):**  

```
the cat in the hat
```

From this text, we want to build the vocabulary for a BPE tokenizer.

---

### **Iteration 1**

#### **Identify Frequent Pairs**  
- In this text, `"th"` appears **twice** (at the beginning and before the second `"e"`).

#### **Replace and Record**  
- Replace `"th"` with a new token ID that is not already in use, e.g., `256`.
- The new text becomes:

  ```
  <256>e cat in <256>e hat
  ```

- The updated vocabulary:

  ```
  0: ...
  ...
  256: "th"
  ```

---

### **Iteration 2**

#### **Identify Frequent Pairs**  
- In the text `<256>e cat in <256>e hat`, the pair `<256>e` appears **twice**.

#### **Replace and Record**  
- Replace `<256>e` with a new token ID that is not already in use, e.g., `257`.
- The new text becomes:

  ```
  <257> cat in <257> hat
  ```

- The updated vocabulary:

  ```
  0: ...
  ...
  256: "th"
  257: "<256>e"
  ```

---

### **Iteration 3**

#### **Identify Frequent Pairs**  
- In the text `<257> cat in <257> hat`, the pair `<257> ` appears **twice** (once at the beginning and once before `"hat"`).

#### **Replace and Record**  
- Replace `<257> ` with a new token ID that is not already in use, e.g., `258`.
- The new text becomes:

  ```
  <258>cat in <258>hat
  ```

- The updated vocabulary:

  ```
  0: ...
  ...
  256: "th"
  257: "<256>e"
  258: "<257> "
  ```

And so forth...

---

### 1.4.2 Concrete Example of the Decoding Part (Step 3)

To restore the original text, we **reverse the process** by substituting each token ID with its corresponding pair in the reverse order they were introduced.

#### **Starting with the final compressed text:**  
```
<258>cat in <258>hat
```

#### **Substitutions (in reverse order):**
1. Substitute `<258> → <257> `  
   ```
   <257> cat in <257> hat
   ```
2. Substitute `<257> → <256>e`  
   ```
   <256>e cat in <256>e hat
   ```
3. Substitute `<256> → "th"`  
   ```
   the cat in the hat
   ```

